# 12.11 & 13.11. RAG Chatbot 📚

📍 [Download notebook](https://github.com/maxschmaltz/Course-LLM-based-Assistants/tree/main/llm-based-assistants/sessions/block2_core/1211_1311/1211_1311.ipynb)

In today'l lab, we will be expanding the chatbot we created in our [previous session](../0611.ipynb). We'll implement a RAG functionality so that the chatbot has access to custom knowledge. In the first part (12.11), we'll preprocess our data for further retrieval. Next day (13.11), we will complete the RAG chatbot and use the data we have preprocessed in the first part to inject our custom knowledge to the LLM. 

Our plan for 12.11 & 13.11:

* [Data Preprocessing](#data)
* [Prompting al LangChain](#prompts)
* [Recap: Basic Chatbot](#recap)
* [Simple RAG](#rag)
* [Advanced RAG](#adv_rag)

## Prerequisites

To start with the tutorial, complete the steps [Prerequisites](../../../../infos/llm_inference_guide/README.md#prerequisites), [Environment Setup](../../../../infos/llm_inference_guide/README.md#environment-setup), and [Getting API Key](../../../../infos/llm_inference_guide/README.md#getting-api-key) from the [LLM Inference Guide](../../../../infos/llm_inference_guide/README.md).

Today, we have more packages so we'll use the requirements file to install the dependencies:

In [1]:
!curl -o requirements.txt https://raw.githubusercontent.com/maxschmaltz/Course-LLM-based-Assistants/main/llm-based-assistants/sessions/block2_core/1211_1311/requirements.txt
!pip install -r requirements.txt

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   118  100   118    0     0    708      0 --:--:-- --:--:-- --:--:--   706

[notice] A new release of pip is available: 24.2 -> 25.3
[notice] To update, run: pip install --upgrade pip


Finally, download the data we'll be working with:

In [2]:
!curl -o topic_overview.pdf https://raw.githubusercontent.com/maxschmaltz/Course-LLM-based-Assistants/main/llm-based-assistants/sessions/block2_core/1211_1311/topic_overview.pdf

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  423k  100  423k    0     0  2292k      0 --:--:-- --:--:-- --:--:-- 2302k


<h2 id="data">1. Data Preprocessing 📕</h2>

As you remember from the lecture, the first step to RAG is data preprocessing. That includes:
1. Loading: load the source (document, website etc.) as a text.
2. Chunking: chunk the loaded text onto smaller pieces.
3. Converting to embeddings: embed the chunks into dense vector for further similarity search.
4. Indexing: put the embeddings into a so-called index -- a special database for efficient storage and search of vectors.

### Loading

We will take a PDF version of the Topic Overview for the last iteration of this course (the current version does not convert nicely). No LLM can know the contents of it, especially some highly specific facts such as dates or key points.

One of ways to load a PDF is to use [`PyPDFLoader`](https://docs.langchain.com/oss/python/integrations/document_loaders/pypdfloader) that load simple textual PDFs and their metadata. In this tutorial, we focus on a simpler variant when there are no multimodal data in the PDF. You can find out more about advanced loading on the PyPDFLoader [LangChain page](https://docs.langchain.com/oss/python/integrations/document_loaders/pypdfloader).

In [3]:
from langchain_community.document_loaders import PyPDFLoader

/Users/maksimshmalts/Documents/Course-LLM-based-Assistants/llm-based-assistants/sessions/block2_core/1211_1311/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
file_path = "./topic_overview.pdf"

In [5]:
loader = PyPDFLoader(file_path)
docs = loader.load()

Ignoring wrong pointing object 10 0 (offset 0)
Ignoring wrong pointing object 31 0 (offset 0)


This function returns a list of `Document` objects, each containing the text of the PDF and its metadata such as title, page, creation date etc.

In [6]:
docs

[Document(metadata={'producer': 'macOS Version 12.7.6 (Build 21H1320) Quartz PDFContext', 'creator': 'Safari', 'creationdate': "D:20250512152829Z00'00'", 'title': 'Topics Overview - LLM-based Assistants', 'moddate': "D:20250512152829Z00'00'", 'source': './topic_overview.pdf', 'total_pages': 12, 'page': 0, 'page_label': '1'}, page_content='12.05.25, 17:28Topics Overview - LLM-based Assistants\nPage 1 of 12https://maxschmaltz.github.io/Course-LLM-based-Assistants/infos/topic_overview.html\nTo p i c s  O v e r v i e wThe schedule is preliminary and subject to changes!\nThe reading for each lecture is given as references to the sources the respective lectures base on. Youare not obliged to read anything. However, you are strongly encouraged to read references marked bypin emojis \n: those are comprehensive overviews on the topics or important works that are beneficialfor a better understanding of the key concepts. For the pinned papers, I also specify the pages span foryou to focus on the 

In [7]:
print(docs[0].page_content)

12.05.25, 17:28Topics Overview - LLM-based Assistants
Page 1 of 12https://maxschmaltz.github.io/Course-LLM-based-Assistants/infos/topic_overview.html
To p i c s  O v e r v i e wThe schedule is preliminary and subject to changes!
The reading for each lecture is given as references to the sources the respective lectures base on. Youare not obliged to read anything. However, you are strongly encouraged to read references marked bypin emojis 
: those are comprehensive overviews on the topics or important works that are beneficialfor a better understanding of the key concepts. For the pinned papers, I also specify the pages span foryou to focus on the most important fragments. Some of the sources are also marked with a popcornemoji 
: that is misc material you might want to take a look at: blog posts, GitHub repos, leaderboardsetc. (also a couple of LLM-based games). For each of the sources, I also leave my subjectiveestimation of how important this work is for this specific topic: from yel

As you can see, the result is not satisfying because the PDF has a more complex structure than just one-paragraph text. To handle it's layout, we could use `UnstructuredLoader`-like OCR engines that will return a `Document` not for the whole page but for a single structure. One of open-source solutions is [Docling](https://docs.langchain.com/oss/python/integrations/document_loaders/docling).

In [8]:
from langchain_docling import DoclingLoader
from langchain_docling.loader import ExportType

In [9]:
loader = DoclingLoader(file_path)
docs = loader.load()

2025-11-11 17:38:58,141 - INFO - detected formats: [<InputFormat.PDF: 'pdf'>]
2025-11-11 17:38:58,182 - INFO - Going to convert document batch...
2025-11-11 17:38:58,183 - INFO - Initializing pipeline for StandardPdfPipeline with options hash 44ae89a68fc272bc7889292e9b5a1bad
2025-11-11 17:38:58,189 - INFO - Loading plugin 'docling_defaults'
2025-11-11 17:38:58,190 - WARNING - The plugin langchain_docling will not be loaded because Docling is being executed with allow_external_plugins=false.
2025-11-11 17:38:58,190 - INFO - Registered picture descriptions: ['vlm', 'api']
2025-11-11 17:38:58,195 - INFO - Loading plugin 'docling_defaults'
2025-11-11 17:38:58,197 - WARNING - The plugin langchain_docling will not be loaded because Docling is being executed with allow_external_plugins=false.
2025-11-11 17:38:58,197 - INFO - Registered ocr engines: ['auto', 'easyocr', 'ocrmac', 'rapidocr', 'tesserocr', 'tesseract']
2025-11-11 17:38:58,572 - INFO - Auto OCR model selected ocrmac.
2025-11-11 17

Look at how Docling parsed a single structure (left sidebar) into a separate `Document`:

In [10]:
print(docs[0].page_content)

LLM-based Assistants
Search
INFOS AND STUFF
Topics Overview
Debates
Pitches
LLM Inference Guide
BLOCK 1: INTRO
22.04. LLMs as a Form of Intelligence vs LLMs as Statistical Machines
24.04. LLM & Agent Basics
29.04. Intro to LangChain
BLOCK 2: CORE TOPICS | PART 1: BUSINESS APPLICATIONS
06.05. Virtual Assistants Pt. 1: Chatbots
08.05. Basic LLM-based Chatbot
BLOCK 2: CORE TOPICS | PART 2: APPLICATIONS IN SCIENCE
Under development
BLOCK 3: WRAP-UP
Under development


However, it also parsed every paragraph from the body of the PDF into a separate `Document`, which destroyed the links between the paragraphs of the same sections: 

In [11]:
print(docs[5].page_content)

Key points :
- Course introduction
- Different perspectives on the nature of LLMs
- Similarities and differences between human and artificial intelligence
- LLMs' (self-)awareness


To avoid such a fine chunking, we ask the engine to output the whole PDF as a single MD file and then chunk it ourselves.

In [12]:
loader = DoclingLoader(file_path, export_type=ExportType.MARKDOWN)
docs = loader.load()

2025-11-11 17:39:03,135 - INFO - detected formats: [<InputFormat.PDF: 'pdf'>]
2025-11-11 17:39:03,136 - INFO - Going to convert document batch...
2025-11-11 17:39:03,136 - INFO - Initializing pipeline for StandardPdfPipeline with options hash 44ae89a68fc272bc7889292e9b5a1bad
2025-11-11 17:39:03,136 - INFO - Auto OCR model selected ocrmac.
2025-11-11 17:39:03,137 - INFO - Accelerator device: 'mps'
2025-11-11 17:39:03,792 - INFO - Accelerator device: 'mps'
2025-11-11 17:39:03,997 - INFO - Processing document topic_overview.pdf
2025-11-11 17:39:06,821 - INFO - Finished converting document topic_overview.pdf in 3.69 sec.


In [13]:
len(docs)

1

In [14]:
print(docs[0].page_content)

## LLM-based Assistants

Search

INFOS AND STUFF

Topics Overview

Debates

Pitches

LLM Inference Guide

BLOCK 1: INTRO

22.04. LLMs as a Form of Intelligence vs LLMs as Statistical Machines

24.04. LLM &amp; Agent Basics

29.04. Intro to LangChain

BLOCK 2: CORE TOPICS | PART 1: BUSINESS APPLICATIONS

06.05. Virtual Assistants Pt. 1: Chatbots

08.05. Basic LLM-based Chatbot

BLOCK 2: CORE TOPICS | PART 2: APPLICATIONS IN SCIENCE

Under development

BLOCK 3: WRAP-UP

Under development

## Topics Overview

The schedule is preliminary and subject to changes !

The reading for each lecture is given as references to the sources the respective lectures base on. You are not obliged to read anything. However, you are strongly encouraged to read references marked by pin emojis : those are comprehensive overviews on the topics or important works that are beneficial for a better understanding of the key concepts. For the pinned papers, I also specify the pages span for you to focus on the most 

### Chunking

During RAG, relevant documents are usually retrieved by semantic similarity that is calculated between the search query and each document in the index. However, if we calculate vectors for the entire PDF pages, we risk not to capture any meaning in the embedding because the context is just too long. That is why usually, loaded text is _chunked_ in a RAG application; embeddings for smaller pieces of text are more discriminative, and thus the relevant context may be retrieved more reliably. Furthermore, it ensure process consistency when working documents of varying sizes.

Different approaches to chunking are described in tutorial [Text splitters](https://docs.langchain.com/oss/python/integrations/splitters/index#text-splitters) from LangChain. Even though Docling returned a MD document, we won't be using the dedicated [`MarkdownHeaderTextSplitter`](https://docs.langchain.com/oss/python/integrations/splitters/markdown_header_metadata_splitter); instead, for a more general picture, we choose the [`RecursiveCharacterTextSplitter`](https://docs.langchain.com/oss/python/integrations/splitters/recursive_text_splitter) -- a good option in terms of simplicity-quality ratio for simple cases. This splitter tries to keep text structures (paragraphs, sentences) together and thus maintain text coherence in chunks.

In [15]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_core.documents import Document
from typing import List

In [16]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=700, # maximum number of characters in a chunk
    chunk_overlap=250, # number of characters to overlap between chunks
    separators=["\n"]
)

def split_page(doc: Document) -> List[Document]:
    chunks = text_splitter.split_text(doc.page_content)
    return [
        Document(
            page_content=chunk,
            metadata={
                **doc.metadata,
                "chunk_n": i
            },
        ) 
        for i, chunk in enumerate(chunks)
    ]

In [17]:
chunks = []
for doc in docs:
    chunks += split_page(doc)

print(f"Converted {len(docs)} pages into {len(chunks)} chunks.")

Converted 1 pages into 51 chunks.


In [18]:
print(chunks[12].page_content)

## Additional Reading :

- Training language models to follow instructions with human feedback, OpenAI
- Training a Helpful and Harmless Assistant with Reinforcement Learning from Human Feedback, Anthropic
- A Survey on the Memory Mechanism of Large Language Model based Agents, Renmin University of China &amp; Huawei Noah's Ark Lab
- Augmenting Language Models with Long-Term Memory, UC Santa Barbara &amp; Microsoft Research
- From LLM to Conversational Agent: A Memory Enhanced Architecture with Fine-Tuning of Large Language Models, Beike Inc.
- Automatic Prompt Selection for Large Language Models, Cinnamon AI, Hung Yen University of Technology and Education &amp; Deakin University


### Convert to Embeddings

As discussed, the retrieval usually supported by vector similarity and so the index contains not the actual texts but their vector representations. Vector representations are created by _embedding models_ -- models usually made specifically for this objective by being trained to create more similar vectors for more similar sentences and to push apart dissimilar sentences in the vector space.

We will use the [`nv-embedqa-e5-v5`](https://build.nvidia.com/nvidia/nv-embedqa-e5-v5?snippet_tab=LangChain) model -- a model from NVIDIA pretrained for English QA.

In [19]:
from langchain_nvidia_ai_endpoints import NVIDIAEmbeddings

In [20]:
EMBEDDING_NAME = "nvidia/nv-embedqa-e5-v5"

embeddings = NVIDIAEmbeddings(
    model=EMBEDDING_NAME, 
    # api_key=os.getenv("NVIDIA_API_KEY")
)

An embedding model receives an input text and returns a dense vector that is believed to capture its semantic properties.

In [21]:
test_embedding = embeddings.embed_query("retrieval augmented generation")
test_embedding

[-0.0036773681640625,
 -0.043121337890625,
 0.047119140625,
 0.0009489059448242188,
 0.0088958740234375,
 0.012298583984375,
 -0.032257080078125,
 -0.035064697265625,
 0.049346923828125,
 -0.06866455078125,
 0.0357666015625,
 -0.0167083740234375,
 0.0596923828125,
 -0.019317626953125,
 0.004131317138671875,
 0.0254974365234375,
 -0.005035400390625,
 0.0222320556640625,
 0.03277587890625,
 -0.03277587890625,
 -0.01904296875,
 0.0172882080078125,
 0.01319122314453125,
 -0.027923583984375,
 0.0295562744140625,
 -0.018035888671875,
 0.01561737060546875,
 -0.0239410400390625,
 -0.00791168212890625,
 0.06072998046875,
 0.00841522216796875,
 0.014068603515625,
 0.03802490234375,
 0.046722412109375,
 0.0213165283203125,
 -0.01702880859375,
 -0.00449371337890625,
 -0.011322021484375,
 -0.02984619140625,
 -0.01305389404296875,
 -0.029876708984375,
 0.00072479248046875,
 -0.0187530517578125,
 0.0518798828125,
 -0.00241851806640625,
 0.00720977783203125,
 -0.049896240234375,
 -0.035400390625,
 -0.

### Indexing

Now that we have split our data and initialized the embeddings, we can start indexing it. There are a lot of different implementations of indexes, you can take a lot at available options in [Vector stores](https://docs.langchain.com/oss/python/integrations/vectorstores). One of the popular choices is [Qdrant](https://docs.langchain.com/oss/python/integrations/vectorstores/qdrant) that provides a simple data management and can be deployed both locally, on a remote machine, and on the cloud.

Qdrant support persisting your vector storage, i.e. storing it on the working machine, but for simplicity, we will use it in the in-memory mode, so that the storage exists only as long as the notebook does.

In [22]:
from langchain_qdrant import QdrantVectorStore
from qdrant_client import QdrantClient
from qdrant_client.http.models import Distance, VectorParams
from uuid import uuid4

First things first, we need to create a _client_ -- a Qdrant instance that will be the entrypoint for all the actions we do with the data.

In [23]:
qd_client = QdrantClient(":memory:")    # in-memory Qdrant client

Then, as we use an in-memory client that does not store the index between the notebook sessions, we need to initialize a _collection_. Alternatively, if we were persisting the data, we would perform a check if the collection exists and then either create or load it.

For Qdrant to initialize the structure of the index correctly, we need to provide the dimentionality of the embedding we will be using as well as teh distance metric.

In [24]:
collection_name = "1211_1311"

qd_client.create_collection(
    collection_name=collection_name,
    # embedding params here
    vectors_config=VectorParams(
        size=len(test_embedding),   # is there a better way?
        distance=Distance.COSINE    # cosine distance
    )
)

True

Finally, we use a LangChain wrapper to connect to the index to unify the workflow.

In [25]:
vector_store = QdrantVectorStore(
    client=qd_client,
    collection_name=collection_name,
    embedding=embeddings
)

Now we are ready to add our chunks to the vector storage. As we will be adding the chunks, the index will take care about converting our passages into embeddings.

In order to be able to delete / modify the chunks afterwards, we assign them with unique ids that we generate dynamically.

In [26]:
ids = [str(uuid4()) for _ in range(len(chunks))]
vector_store.add_documents(
    chunks,
    ids=ids
)

['7a7f60df-fb9e-4230-9d5f-bb33d7c2e458',
 '1676dfe2-1257-4184-8a31-c561b3c3b640',
 '644af39d-302d-45de-b249-9685af723cf4',
 'c03cf17b-5ad2-49d9-a70f-0971e52dd849',
 '5857f6d5-3a6a-457a-a80f-78bc53221c57',
 'e6ffc86c-e81a-46d3-93b0-e85d9d8ca6a4',
 '0098a242-e389-4ba3-813a-d1e15f582695',
 '720ae4c8-a7dc-4d54-9241-412c5c0938ca',
 'dd33f559-e593-4759-ac0c-a017bf423f8f',
 'bb8a423f-4642-4138-99d8-4430047a93b2',
 '1f272c95-f4e9-4de5-b5f5-8fdc394bc683',
 '02591385-401d-4023-a3f1-37d2ef892c89',
 'd506a603-e4ce-46d7-b2cd-a4f5746e2e07',
 '88778634-9e87-4b20-840b-5821d9fff85f',
 '94571223-60b1-4d1a-a541-8c566a548e5c',
 '672388aa-567f-41f5-b48a-0452cd863cb4',
 '9d6af252-e9d8-4995-8573-cfe69ab347d0',
 '762136ad-16d2-4af6-89bf-5f97e38c2c50',
 '9ba9dbf6-8328-4f5a-adc6-2c747842adf3',
 'c6f257a1-06db-402f-833b-36ffbcb279db',
 '46358de2-952f-439b-9c76-fa48a0e2ca00',
 '97531f90-82bc-42bd-a92a-1ee881694f92',
 '9beca309-8c29-4204-ae68-e5a3b73c4a31',
 '9e7d00ea-3bd4-454e-bdae-4161468cc896',
 'ce9a7a47-5efc-

In [27]:
vector_store.get_by_ids([ids[-1]])

[Document(metadata={'source': './topic_overview.pdf', 'chunk_n': 50, '_id': '1eaba0b9-7b5e-4201-bad5-cc01cd9a9e66', '_collection_name': '1211_1311'}, page_content="Reading : see session 01.07 and session 03.07\n\n## 24.07. Debate : Role of AI in Recent Years + Wrap-up\n\nOn material of session 17.07\n\nThe course will be concluded by the final debates, after which a short Q&amp;A session will be held. Copyright © 2025, Maksim Shmalts Made with Sphinx and @pradyunsg's Furo\n\nDebate topics:\n\n- LLM Behavior: Evidence of Awareness or Illusion of Understanding ?\n- Should We Limit the Usage of AI ?\n\nReading : see session 17.07")]

In [28]:
vector_store.delete([ids[-1]])
vector_store.get_by_ids([ids[-1]])

[]

The index provides the necessary functionality for the query-based retrieval:

In [29]:
vector_store.similarity_search("retrieval augmented generation", k=3)

[Document(metadata={'source': './topic_overview.pdf', 'chunk_n': 15, '_id': '672388aa-567f-41f5-b48a-0452cd863cb4', '_collection_name': '1211_1311'}, page_content="## Key points :\n\n- General knowledge vs context\n- Knowledge indexing, retrieval &amp; ranking\n- Retrieval tools\n- Agentic RAG\n\n## Core Reading :\n\n- Retrieval Augmented Generation or Long-Context LLMs ? A Comprehensive Study and Hybrid Approach (pages 1-7), Google DeepMind &amp; University of Michigan\n- A Survey on Retrieval-Augmented Text Generation for Large Language Models (sections 1-7), York University\n\n## Additional Reading :\n\n- Don't Do RAG: When Cache-Augmented Generation is All You Need for Knowledge Tasks, National Chengchi University &amp; Academia Sinica"),
 Document(metadata={'source': './topic_overview.pdf', 'chunk_n': 16, '_id': '9d6af252-e9d8-4995-8573-cfe69ab347d0', '_collection_name': '1211_1311'}, page_content="## Additional Reading :\n\n- Don't Do RAG: When Cache-Augmented Generation is All Y

<h2 id="prompts">2. Prompting al LangChain 📝</h2>

When you build more complex algorithms, just passing the human query directly might be not enough. Sometimes, you need to give more specific instructions, pre- and append additional stuff to the messages, or just accept the input in a more flexible way. For that, you can use `ChatPromptTemplate` that allows for reusability and flexibility when processing inputs.

The key idea is simple: in a `ChatPromptTemplate`, you write all the constant fragments in plain text and then use placeholders to mark the places where some variable parts will be added. Then, when you receive an input, LangChain fills the placeholders and you receive the desired version of the message with all the placeholders filled automatically.

In [ ]:
from langchain_core.prompts import ChatPromptTemplate, SystemMessagePromptTemplate, MessagesPlaceholder

For example, let us make a template that will surround the user query with some specific instructions for CoT prompting.

In [ ]:
input_template_str = """\
The user is asking a question. Please answer it using step-by-step reasoning. \
On each reasoning step, assess whether this reasoning step is good or not, \
on a scale from 1 to 10.

The user question is:

============
{input}
"""

input_template = ChatPromptTemplate.from_template(input_template_str)

Now, even though the user will provide a simple query as usual, the LLM will receive all the additional instructions you wrote. A `ChatPromptTemplate` uses **keys** to fill the placeholders so you should pass it a corresponding `dict`.

In [ ]:
example = input_template.invoke(
    {
        "input": "How big is the distance between the Earth and the Moon?"
    }
)

example

ChatPromptValue(messages=[HumanMessage(content='The user is asking a question. Please answer it using step-by-step reasoning. On each reasoning step, assess whether this reasoning step is good or not, on a scale from 1 to 10.\n\nThe user question is:\n\n============\nHow big is the distance between the Earth and the Moon?\n', additional_kwargs={}, response_metadata={})])

In [ ]:
print(example.messages[0].content)

The user is asking a question. Please answer it using step-by-step reasoning. On each reasoning step, assess whether this reasoning step is good or not, on a scale from 1 to 10.

The user question is:

How big is the distance between the Earth and the Moon?



You can also make prompt templates of a higher level -- that is, not for a single message, but for an entire sequence of messages. To do so, you need to nest `ChatPromptTemplate`s for separate messages and use `MessagesPlaceholder` for sequences. This approach gives you a universal way to fill the placeholders, be it a separate fragment of a certain message or a whole sequence of messages: all you need is to be careful with the keys, and LangChain will take care of the rest.

In [ ]:
system_template = SystemMessagePromptTemplate.from_template("Answer in the following language: {language}.")

prompt_template = ChatPromptTemplate.from_messages(
    [
        system_template,
        MessagesPlaceholder(variable_name="messages")   # here, you add an entire sequence of messages
    ]
)

Alternative: pass separate messages as pairs of raw strings where the first string describes the role (`"system"`, `"user"`, `"ai"`) and the second -- the content.

In [ ]:
prompt_template = ChatPromptTemplate.from_messages(
    [
        ("system", "Answer in the following language: {language}."),    # here, you modify a fragment of the system message
        MessagesPlaceholder(variable_name="messages")   # here, you add an entire sequence of messages
    ]
)

In [ ]:
prompt_template.invoke({
    "language": "Spanish",
    "messages": example.to_messages()
})

ChatPromptValue(messages=[SystemMessage(content='Answer in the following language: Spanish.', additional_kwargs={}, response_metadata={}), HumanMessage(content='The user is asking a question. Please answer it using step-by-step reasoning. On each reasoning step, assess whether this reasoning step is good or not, on a scale from 1 to 10.\n\nThe user question is:\n\n============\nHow big is the distance between the Earth and the Moon?\n', additional_kwargs={}, response_metadata={})])

We can now incorporate this logic into our chatbot.

In [ ]:
class CoTChatbot(Chatbot):
    
    def __init__(self, llm):
        super().__init__(llm)
        self.input_template = input_template
        self.prompt_template = prompt_template

    def _input_node(self, state: State) -> dict:
        user_query = input("Your message: ")
        if user_query != "quit":
            # invoke the template here
            human_message = self.input_template.invoke(
                {
                    "input": user_query
                }
            ).to_messages()
        else:
            human_message = HumanMessage(content=user_query)
        n_turns = state["n_turns"]
        # add the input to the messages
        return {
            "messages": human_message,
            "n_turns": n_turns + 1
        }
    
    def _respond_node(self, state: State) -> dict:
        # invoke the template here;
        # since the state is already a dictionary, we can just pass it as is
        prompt = self.prompt_template.invoke(state)
        n_turns = state["n_turns"]
        response = self.llm.invoke(prompt)
        # add the response to the messages
        return {
            "messages": response,
            "n_turns": n_turns + 1
        }

    def run(self, language):
        # since the system message is now part of the prompt template,
        # we don't need to add it to the input
        input = {
            "messages": [],
            "n_turns": 0,
            "language": language
        }
        for event in self.chatbot.stream(input, stream_mode="values"):
            if event["messages"]:
                event["messages"][-1].pretty_print()
                print("\n")

In [ ]:
cot_chatbot = CoTChatbot(llm)

In [ ]:
cot_chatbot.run("German")

================================ Human Message =================================

The user is asking a question. Please answer it using step-by-step reasoning. On each reasoning step, assess whether this reasoning step is good or not, on a scale from 1 to 10.

The user question is:

What is the most probable year for the AGI to come?



================================== Ai Message ==================================

Um Ihre Frage zu beantworten, werde ich eine Schritt-für-Schritt-Analyse durchführen.

Schritt 1: Definition von AGI
Ich muss zunächst definieren, was AGI (Artificial General Intelligence) bedeutet. AGI bezeichnet eine künstliche Intelligenz, die in der Lage ist, alle intellektuellen Aufgaben zu erledigen, die auch ein Mensch erledigen kann. (Gute Bewertung: 8/10, da die Definition ziemlich allgemein ist und je nach Kontext variieren kann)

Schritt 2: Aktueller Stand der KI-Forschung
Als nächstes muss ich den aktuellen Stand der KI-Forschung betrachten. Die KI-Forschung ha

<h2 id="recap">3. Recap: Basic Chatbot 🤖</h2>

In the [last session](../0805/0805.ipynb), we created a chatbot with LangGraph that has three nodes:
1. The input receival node. It prompted the user for the input and stored it in the messages for further interaction with the LLM.
2. The router node. It performed the check whether the user wants to exit.
3. The chatbot node. It received the input if the user had not quit, passed it to the LLM, and returned the generation.

Each node is a Python function that (typically) accepts the single argument: the state. To update the state, the function should return a `dict` with the keys corresponding to the state keys, with the **updated** values. The update behavior depends on how you defined your state class (will be rewritten by default or processed by a function if given in `Annotated`).

In [1]:
from langchain_core.messages import SystemMessage, HumanMessage
from langchain_nvidia_ai_endpoints import ChatNVIDIA
from langchain_core.rate_limiters import InMemoryRateLimiter

In [2]:
# read system variables
import os
import dotenv

dotenv.load_dotenv()    # that loads the .env file variables into os.environ

True

In [188]:
# choose any model, catalogue is available under https://build.nvidia.com/models
MODEL_NAME = "meta/llama-3.3-70b-instruct"

# this rate limiter will ensure we do not exceed the rate limit
# of 40 RPM given by NVIDIA
rate_limiter = InMemoryRateLimiter(
    requests_per_second=30 / 60,  # 30 requests per minute to be sure
    check_every_n_seconds=0.1,  # wake up every 100 ms to check whether allowed to make a request,
    max_bucket_size=4,  # controls the maximum burst size
)

llm = ChatNVIDIA(
    model=MODEL_NAME,
    api_key=os.getenv("NVIDIA_API_KEY"), 
    temperature=0,   # ensure reproducibility,
    rate_limiter=rate_limiter  # bind the rate limiter
)

In [4]:
from typing import Annotated, List
from typing_extensions import TypedDict
from langchain_core.messages import BaseMessage
from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages
from langchain_core.runnables.graph import MermaidDrawMethod

In [5]:
import nest_asyncio
nest_asyncio.apply()  # this is needed to draw the PNG in Jupyter

State scheme describes what structure the state should be of.

In [6]:
class State(TypedDict):
    # `messages` is a list of messages of any kind. The `add_messages` function
    # in the annotation defines how this state key should be updated
    # (in this case, it appends messages to the list, rather than overwriting them)
    messages: Annotated[List[BaseMessage], add_messages]
    # Since we didn't define a function to update it, it will be rewritten at each transition
    # with the value you provide
    n_turns: int    # just for demonstration
    language: str    # just for demonstration

In [7]:
class Chatbot:

    _graph_path = "./graph.png"
    
    def __init__(self, llm):
        self.llm = llm
        self._build()
        self._display_graph()

    def _build(self):
        # graph builder
        self._graph_builder = StateGraph(State)
        # add the nodes
        self._graph_builder.add_node("input", self._input_node)
        self._graph_builder.add_node("respond", self._respond_node)
        # define edges
        self._graph_builder.add_edge(START, "input")
        self._graph_builder.add_conditional_edges("input", self._is_quitting_node, {False: "respond", True: END})
        self._graph_builder.add_edge("respond", "input")
        # compile the graph
        self._compile()

    def _compile(self):
        self.chatbot = self._graph_builder.compile()

    def _input_node(self, state: State) -> dict:
        user_query = input("Your message: ")
        human_message = HumanMessage(content=user_query)
        n_turns = state["n_turns"]
        # add the input to the messages
        return {
            "messages": human_message,   # this will append the input to the messages
            "n_turns": n_turns + 1,  # and this will rewrite the number of turns
            # "language": ...  # we don't update this field so we just leave it out
        }
    
    def _respond_node(self, state: State) -> dict:
        messages = state["messages"]    # will already contain the user query
        n_turns = state["n_turns"]
        response = self.llm.invoke(messages)
        # add the response to the messages
        return {
            "messages": response,   # this will append the response to the messages
            "n_turns": n_turns + 1,  # and this will rewrite the number of turns
            # "language": ...  # we don't update this field so we just leave it out
        }
    
    def _is_quitting_node(self, state: State) -> dict:
        # check if the user wants to quit
        user_message = state["messages"][-1].content
        return user_message.lower() == "quit"
    
    def _display_graph(self):
        # unstable
        try:
            self.chatbot.get_graph().draw_mermaid_png(
                draw_method=MermaidDrawMethod.PYPPETEER,
                output_file_path=self._graph_path
            )
        except Exception as e:
            pass

    # add the run method
    def run(self):
        input = {
            "messages": [
                SystemMessage(
                    content="You are a helpful and honest assistant." # role
                )
            ],
            "n_turns": 0,
            "language": "some_value"
        }
        for event in self.chatbot.stream(input, stream_mode="values"):   #stream_mode="updates"):
            for key, value in event.items():
                print(f"{key}:\t{value}")
            print("\n")

In [8]:
chatbot = Chatbot(llm)

In [15]:
chatbot.run()

messages:	[SystemMessage(content='You are a helpful and honest assistant.', additional_kwargs={}, response_metadata={}, id='b9118749-ab3b-4c52-a513-698ea619b9e5')]
n_turns:	0
language:	some_value


messages:	[SystemMessage(content='You are a helpful and honest assistant.', additional_kwargs={}, response_metadata={}, id='b9118749-ab3b-4c52-a513-698ea619b9e5'), HumanMessage(content='hi, tell me a joke', additional_kwargs={}, response_metadata={}, id='1783ee73-fb45-4542-ba45-9a534074c50e')]
n_turns:	1
language:	some_value


messages:	[SystemMessage(content='You are a helpful and honest assistant.', additional_kwargs={}, response_metadata={}, id='b9118749-ab3b-4c52-a513-698ea619b9e5'), HumanMessage(content='hi, tell me a joke', additional_kwargs={}, response_metadata={}, id='1783ee73-fb45-4542-ba45-9a534074c50e'), AIMessage(content="Here's one:\n\nWhat do you call a fake noodle?\n\nAn impasta!\n\nHope that made you smile! Do you want to hear another one?", additional_kwargs={}, response_me

<h2 id="rag">4. Simple RAG 💉</h2>

The basic RAG workflow is pretty straightforward: we just retrieve _k_ most relevant documents and them insert them into the prompt as a part of the context.

For that, we will combine the skills we have obtained so far to build a LangGraph agent that receives the input, checks if the user wants to quit, then do the retrieval and generate a context-aware response if not. We will build on the basic version of our first chatbot; to add the RAG functionality, we need to add a retrieval node and modify the generation prompt to inject the retrieved documents.

First, we need to initialize our LLM.

In [ ]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.tools.retriever import create_retriever_tool

In [ ]:
# role: restrict it from the parametric knowledge
basic_rag_system_prompt = """\
You are an assistant that has access to a knowledge base. \
You should use the knowledge base to answer the user's questions.
"""


# this will add the context to the input
context_injection_prompt = """\
The user is asking a question. \
You should answer using the following context:

==========================
{context}
==========================


The user question is:
{input}
"""


# finally, gather the system message, the previous messages,
# and the input with the context
basic_rag_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", basic_rag_system_prompt),   # system message
        MessagesPlaceholder(variable_name="messages"),  # previous messages
        ("user", context_injection_prompt)  # user message
    ]
)

LangGraph provides a pre-built tool to conveniently create a retriever tool. As this is basic RAG, we don't generate queries for the retriever for now and just use the user input as the query.

In [ ]:
class BasicRAGChatbot(Chatbot):

    _graph_path = "./graph_basic_rag.png"
    
    def __init__(self, llm, k=5):
        super().__init__(llm)
        self.basic_rag_prompt = basic_rag_prompt
        self.retriever = vector_store.as_retriever(search_kwargs={"k": k})    # retrieve 5 documents
        self.retriever_tool = create_retriever_tool(    # and this is the tool
            self.retriever,
            "retrieve_internal_data",  # name
            "Search relevant information in internal documents.",   # description
        )

    def _build(self):
        # graph builder
        self._graph_builder = StateGraph(SimpleState)
        # add the nodes
        self._graph_builder.add_node("input", self._input_node)
        self._graph_builder.add_node("retrieve", self._retrieve_node)
        self._graph_builder.add_node("respond", self._respond_node)
        # define edges
        self._graph_builder.add_edge(START, "input")
        # basic rag: no planning, just always retrieve
        self._graph_builder.add_conditional_edges("input", self._is_quitting_node, {False: "retrieve", True: END})
        self._graph_builder.add_edge("retrieve", "respond")
        self._graph_builder.add_edge("respond", "input")
        # compile the graph
        self._compile()
    
    def _retrieve_node(self, state: SimpleState) -> dict:
        # retrieve the context
        user_query = state["messages"][-1].content  # use the last message as the query
        context = self.retriever_tool.invoke({"query": user_query})
        # add the context to the messages
        return {
            "messages": context
        }
    
    def _respond_node(self, state: SimpleState) -> dict:
        # the workflow is designed so that the context is always the last message
        # and the user query is the second to last message;
        # finally, we will be combining the context and the user query
        # into a single message so we remove those two from the messages
        context = state["messages"].pop(-1).content
        user_query = state["messages"].pop(-1).content
        prompt = self.basic_rag_prompt.invoke(
            {
                "messages": state["messages"],  # this goes to the message placeholder
                "context": context,  # this goes to the user message
                "input": user_query    # this goes to the user message
            }
        )
        response = self.llm.invoke(prompt)
        # add the response to the messages
        return {
            "messages": response
        }

    def run(self):
        input = {"messages": []}
        for event in self.chatbot.stream(input, stream_mode="values"):
            if event["messages"]:
                event["messages"][-1].pretty_print()
                print("\n")

In [ ]:
basic_rag_chatbot = BasicRAGChatbot(llm)

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [ ]:
basic_rag_chatbot.run()

================================ Human Message =================================

what sessions do I have about virtual assistants?


Adaptive RAG, LangGraph
Multimodality, LangChain
Week 520.05. Lecture: Virtual Assistants Pt. 3: Multi-agent EnvironmentThis lectures concludes the Virtual Assistants cycle and directs its attention to automating everyday /business operations in a multi-agent environment. We’ll look at how agents communicate with eachother, how their communication can be guided (both with and without involvement of a human), andthis all is used in real applications.
Key points:
Multi-agent environment
Human in the loop

Prompt Templates, LangChain
Few-shot prompting, LangChain
Week 413.05. Lecture: Virtual Assistants Pt. 2: RAGContinuing the first part, the second part will expand scope of chatbot functionality and will teach it torefer to custom knowledge base to retrieve and use user-specific information. Finally, the most widelyused deployment methods will be briefly 

As you can see, it already works pretty well, but as the retrieval goes by the user query directly, all the previous context of the conversation is not considered. To handle that, let's add a node that would reformulate the query taking in consideration the previous interaction.

For that, we need an additional prompt.

In [ ]:
# finally, gather the system message, the previous messages,
# and the input with the context
reformulate_query_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "Given the previous conversation, reformulate the user query in the last message to a full question. "
            "Return only the reformulated query, without any other text."
        ),   # system message
        MessagesPlaceholder(variable_name="messages")  # previous messages
    ]
)

In [ ]:
class BasicPlusRAGChatbot(BasicRAGChatbot):

    _graph_path = "./graph_basic_plus_rag.png"

    def __init__(self, llm, k=5):
        super().__init__(llm, k)
        self.reformulate_query_prompt = reformulate_query_prompt

    def _build(self):
        # graph builder
        self._graph_builder = StateGraph(SimpleState)
        # add the nodes
        self._graph_builder.add_node("input", self._input_node)
        self._graph_builder.add_node("reformulate_query", self._reformulate_query_node)
        self._graph_builder.add_node("retrieve", self._retrieve_node)
        self._graph_builder.add_node("respond", self._respond_node)
        # define edges
        self._graph_builder.add_edge(START, "input")
        # basic rag: no planning, just always retrieve
        self._graph_builder.add_conditional_edges("input", self._is_quitting_node, {False: "reformulate_query", True: END})
        self._graph_builder.add_edge("reformulate_query", "retrieve")
        self._graph_builder.add_edge("retrieve", "respond")
        self._graph_builder.add_edge("respond", "input")
        # compile the graph
        self._compile()

    def _reformulate_query_node(self, state: SimpleState) -> dict:
        prompt = self.reformulate_query_prompt.invoke(state)
        generated_query = self.llm.invoke(prompt)
        # since we use the generated query instead of the user query,
        # we need to remove the user query from the messages
        state["messages"].pop(-1)
        return {
            "messages": generated_query # append the generated query to the messages
        }

In [ ]:
basic_plus_rag_chatbot = BasicPlusRAGChatbot(llm, k=7)

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [ ]:
basic_plus_rag_chatbot.run()

================================ Human Message =================================

what do I have about RAG?


================================== Ai Message ==================================

What information do I have about RAG?


================================ Human Message =================================

On material of session 06.05 and session 13.05
The first pitch will be dedicated to a custom RAG chatbot that the contractors (the presentingstudents, see the infos about Pitches) will have prepared to present. The RAG chatbot will have to beable to retrieve specific information from the given documents (not from the general knowledge!) anduse it in its responses. Specific requirements will be released on 22.05.
Reading: see session 06.05, session 08.05, session 13.05, and session 15.05
19.06.Ausfalltermin

Retrieval tools
Agentic RAG
Core Reading:
 Retrieval Augmented Generation or Long-Context LLMs? A Comprehensive Study and HybridApproach (pages 1-7), Google DeepMind & Unive

<h2 id="adv_rag">5. Advanced RAG 😎</h2>

Now we can move to a more complicated implementation. We will no make an iterative RAG chatbot: this chatbot will retrieve contexts iteratively and decide at each step whether the chunks retrieved so far are sufficient to answer the question; the answer is generated only when the retrieved contexts are enough.

Basically, we have almost everything we need to implement an iterative RAG pipeline. We only need to add three more nodes:
1. A node to generate search queries for the index: now we won't use the user query but specifically generate queries for the index.
2. A decision node, in which the LLM will decide whether the context retrieved so far is enough to proceed to generation of the response.
3. Query transformer that will reformulate the query to retrieve further chunks when it's needed.

As a useful addition, we will also add LLM-based filtering of the retrieved documents to filter out the documents that are semantically similar to the query but are not really relevant for answering the question.

Thus, we need to add 4 nodes totally.

We will start with the grader that will output a binary score for the relevance: `True`(relevant) or `False` (irrelevant). To implement this functionality, we'll bind the LLM to a true/false structured output.

In [ ]:
from pydantic import BaseModel, Field

In [ ]:
class YesNoVerdict(BaseModel):
    verdict: bool = Field(..., description="Boolean answer to the given binary question.")

We will also need to transform the state to accumulate the contexts gathered so far.

In [ ]:
class AdvancedRAGState(SimpleState): # "messages" is already defined in SimpleRAGState
    contexts: List[List[Document]]    # this is the list of retrieved documents, one list per retrieval

And we also need prompts to filter the documents, to decide whether the contexts are supportive enough, and to transform the query if not.

In [ ]:
generate_query_template = """\
The user is asking a question. \
You have an access to a knowledge base. \
Your task is to generate a query that will retrieve the most relevant documents \
from the knowledge base to answer the user question. \
Return the query only, without any other text.


The user question is:
{input}
"""

generate_query_prompt = ChatPromptTemplate.from_template(generate_query_template)

In [ ]:
context_relevant_template = """\
The user is asking a question. \
For answering the question, you colleague have retrieved the following document:


===========================
{context}
===========================


You task is to assess whether this document is relevant to answer the user question. \
Relevant means that the document contains specific information should be used \
directly to answer the user question. \
Return True if the document is relevant, and False otherwise.


The user question is:
{input}
"""

context_relevant_prompt = ChatPromptTemplate.from_template(context_relevant_template)

In [ ]:
contexts_sufficient_template = """\
The user is asking a question. \

For answering the question, you colleague have retrieved the following document:


===========================
{contexts_str}
===========================


You task is to assess whether the retrieved documents contain an answer the user question. \
Return True if the documents are sufficient, and False otherwise.


The user question is:
{input}
"""

contexts_sufficient_prompt = ChatPromptTemplate.from_template(contexts_sufficient_template)

In [ ]:
transform_query_template = """\
The user is asking a question. \

For answering the question, your colleague the following document has been retrieved:


===========================
{contexts_str}
===========================


To retrieve these documents, the following query has been used:
{query}


However, the query is not very good so the retrieved documents were not helpful. \
Your task is to transform the query into a better one, so that the retrieved documents are more relevant. \
Return the transformed query only, without any other text.


The user question is:
{input}
"""

transform_query_prompt = ChatPromptTemplate.from_template(transform_query_template)

In [ ]:
class IterativeRAGChatbot(BasicPlusRAGChatbot):

    _graph_path = "./graph_iterative_rag.png"

    def __init__(self, llm, k=5, max_generations=4):
        super().__init__(llm, k)
        self.max_generations = max_generations
        self.boolean_llm = llm.with_structured_output(YesNoVerdict)
        self.generate_query_prompt = generate_query_prompt
        self.context_relevant_grader = context_relevant_prompt | self.boolean_llm
        self.contexts_sufficient_grader = contexts_sufficient_prompt | self.boolean_llm
        self.transform_query_prompt = transform_query_prompt

    def _build(self):
        # graph builder
        self._graph_builder = StateGraph(AdvancedRAGState)
        # add the nodes
        self._graph_builder.add_node("input", self._input_node)
        self._graph_builder.add_node("reformulate_query", self._reformulate_query_node)
        self._graph_builder.add_node("generate_query", self._generate_query_node)
        self._graph_builder.add_node("retrieve", self._retrieve_node)
        self._graph_builder.add_node("filter_documents", self._filter_documents_node)
        self._graph_builder.add_node("transform_query", self._transform_query_node)
        self._graph_builder.add_node("respond", self._respond_node)
        # define edges
        self._graph_builder.add_edge(START, "input")
        # basic rag: no planning, just always retrieve
        self._graph_builder.add_conditional_edges("input", self._is_quitting_node, {False: "reformulate_query", True: END})
        self._graph_builder.add_edge("reformulate_query", "generate_query")
        self._graph_builder.add_edge("generate_query", "retrieve")
        self._graph_builder.add_edge("retrieve", "filter_documents")
        self._graph_builder.add_conditional_edges(
            "filter_documents",
            self._contexts_sufficient_node,
            {
                False: "transform_query",
                True: "respond",
                None: END   # max generations reached
            }
        )
        self._graph_builder.add_edge("transform_query", "retrieve")
        self._graph_builder.add_edge("respond", "input")
        # compile the graph
        self._compile()

    def _generate_query_node(self, state: AdvancedRAGState) -> dict:    
        user_query = state["messages"][-1].content  # that will be the reformulated user query
        prompt = generate_query_prompt.invoke({"input": user_query})
        search_query = self.llm.invoke(prompt)
        return {
            "messages": search_query
        }

    # now store the contexts in the separate field
    def _retrieve_node(self, state: AdvancedRAGState) -> dict:    
        # retrieve the context
        query = state["messages"][-1].content  # that will be the generated query
        # now use the retriever directly to get a list of documents and not a combined string
        contexts = self.retriever.invoke(query)
        # add the context to the messages
        return {
            "contexts": state["contexts"] + [contexts]  # could have also used `Annotated` here
        }
    
    def _filter_documents_node(self, state: AdvancedRAGState) -> dict:
        query = state["messages"][-1].content  # that will be the generated query
        # since the retrieved documents are graded at the same step,
        # we only need to pass the last batch of documents
        contexts = state["contexts"].pop(-1)  # will be replaced with the filtered ones
        # grade each document separately and only keep the relevant ones
        relevant_contexts = []
        for context in contexts:
            print("Grading document:\n\n", context.page_content)
            verdict = self.context_relevant_grader.invoke(
                {
                    "context": context.page_content,    # this is a Document object
                    "input": query
                }
            )
            print(f"Verdict: {verdict.verdict}")
            print(f"\n\n=====================\n\n")
            if verdict.verdict:    # boolean value according to the Pydantic model
                relevant_contexts.append(context)
        return {
            "contexts": state["contexts"] + [relevant_contexts]  # could have also used `Annotated` here
        }
    
    def _contexts_sufficient_node(self, state: AdvancedRAGState) -> dict:
        query = state["messages"][-2].content   # that will be the reformulated user query, -1 is the generated search query
        all_contexts = state["contexts"]
        # flatten and transform the list of lists into a single list
        contexts = [context for sublist in all_contexts for context in sublist]
        contexts_str = "\n\n".join([context.page_content for context in contexts])
        print("Deciding whether the documents are sufficient")
        verdict = self.contexts_sufficient_grader.invoke(
                {
                "contexts_str": contexts_str,    # this is a Document object
                "input": query
            }
        )
        print(f"Verdict: {verdict.verdict}")
        print(f"\n\n=====================\n\n")
        if not verdict.verdict and len(all_contexts) == self.max_generations:
            return  # will route to END
        return verdict.verdict
    
    def _transform_query_node(self, state: AdvancedRAGState) -> dict:
        # since we will be replacing the user query with the transformed one,
        # we need to remove the old query
        search_query = state["messages"].pop(-1).content   # this is the generated search query
        # the the reformulated user query is the last message
        user_query = state["messages"][-1].content
        all_contexts = state["contexts"]
        # flatten and transform the list of lists into a single list
        contexts = [context for sublist in all_contexts for context in sublist]
        contexts_str = "\n\n".join([context.page_content for context in contexts])
        prompt = self.transform_query_prompt.invoke(
            {
                "contexts_str": contexts_str,
                "query": search_query,
                "input": user_query
            }
        )
        transformed_search_query = self.llm.invoke(prompt)
        return {
            "messages": transformed_search_query   # this will append the transformed query to the messages
        }
    
    def run(self):
        input = {"messages": [], "contexts": [], "suka": 0}
        for event in self.chatbot.stream(input, stream_mode="values"):
            if event["messages"]:
                event["messages"][-1].pretty_print()
                print("\n")

In [ ]:
# make small k to ensure one retrieval is not enough
iterative_rag_chatbot = IterativeRAGChatbot(llm, k=2)

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [ ]:
iterative_rag_chatbot.run()

================================ Human Message =================================

what are the key point of the next lecture after the one on 13.05


================================== Ai Message ==================================

What are the key points of the next lecture after the one scheduled for May 13th?


================================== Ai Message ==================================

lecture schedule May 13th next lecture key points


================================== Ai Message ==================================

lecture schedule May 13th next lecture key points


Grading document:

 12.05.25, 17:28Topics Overview - LLM-based Assistants
Page 1 of 12https://maxschmaltz.github.io/Course-LLM-based-Assistants/infos/topic_overview.html
To p i c s  O v e r v i e wThe schedule is preliminary and subject to changes!
The reading for each lecture is given as references to the sources the respective lectures base on. Youare not obliged to read anything. However, you are strongly enco